In [1]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades
import pandas as pd
import time
from finta import TA
import yfinance as yf
import numpy as np
import pandas as pd
import yfinance as yf
from alpha_vantage.timeseries import TimeSeries
import datetime as dt
import copy
import matplotlib.pyplot as plt


In [2]:
def MACD(DF,a,b,c):
    """function to calculate MACD
       typical values a = 12; b =26, c =9"""
    df = DF.copy()
    df["MA_Fast"]=df["close"].ewm(span=a,min_periods=a).mean()
    df["MA_Slow"]=df["close"].ewm(span=b,min_periods=b).mean()
    df["MACD"]=df["MA_Fast"]-df["MA_Slow"]
    df["Signal"]=df["MACD"].ewm(span=c,min_periods=c).mean()
    df.dropna(inplace=True)
    df.drop(["MA_Fast","MA_Slow"],axis=1)
    return df

In [3]:
client = oandapyV20.API(access_token="5feaa802c52be3e714d4bd74bc1a9169-8cbadb5f1a922e05aad7d56841b3f44a",environment="practice")
account_id = "101-002-19512089-001"
#defining strategy parameters

In [4]:
tickers=["EUR_USD","AUD_USD","GBP_USD","USD_JPY","EUR_JPY"]
investment=5000
risk=50                        

In [5]:
def market_order(instrument,investment,risk):
        account_id = "101-002-19512089-001"

        params = {"instruments": instrument}
        price=pricing.PricingInfo(accountID=account_id,params=params)
        price=float(client.request(price)["prices"][0]["closeoutBid"])
        units=round(investment/price,0)
        sl=round(risk/units,3)

        data = {
                "order": {
                "price": "",
                "stopLossOnFill": {
                "trailingStopLossOnFill": "GTC",
                "distance": str(sl)
                                },
                "timeInForce": "FOK",
                "instrument": str(instrument),
                "units": str(units),
                "type": "MARKET",
                "positionFill": "DEFAULT"
                        }
                }
        r = orders.OrderCreate(accountID=account_id, data=data)
        client.request(r)
        print("order filled for ",instrument)

In [6]:
def market_order1(instrument,investment):
    account_id = "101-002-19512089-001"

    params = {"instruments": instrument}
    price=pricing.PricingInfo(accountID=account_id,params=params)
    price=float(client.request(price)["prices"][0]["closeoutBid"])
    units=round(investment/price,0)

    data = {
                "order": {
                "price": "",
         
                "timeInForce": "FOK",
                "instrument": str(instrument),
                "units": str(units),
                "type": "MARKET",
                "positionFill": "DEFAULT"
                        }
                }
    r = orders.OrderCreate(accountID=account_id, data=data)
    client.request(r)

    print("order squared off for ",instrument)

In [7]:
def market_order2(instrument,investment):
    account_id = "101-002-19512089-001"

    params = {"instruments": instrument}
    price=pricing.PricingInfo(accountID=account_id,params=params)
    price=float(client.request(price)["prices"][0]["closeoutBid"])
    units=round(investment/price,0)

    data = {
                "order": {
                "price": "",
         
                "timeInForce": "FOK",
                "instrument": str(instrument),
                "units": str(units),
                "type": "MARKET",
                "positionFill": "DEFAULT"
                        }
                }
    r = orders.OrderCreate(accountID=account_id, data=data)
    client.request(r)

    print("order squared off for ",instrument)

In [8]:
def candles(instrument):
    params = {"count": 50,"granularity": "M5"} #granularity can be in seconds S5 - S30, minutes M1 - M30, hours H1 - H12, days D, weeks W or months M
    candles = instruments.InstrumentsCandles(instrument=instrument,params=params)
    client.request(candles)
    ohlc_dict = candles.response["candles"]
    ohlc = pd.DataFrame(ohlc_dict)
    ohlc_df = ohlc.mid.dropna().apply(pd.Series)
    ohlc_df["volume"] = ohlc["volume"]
    ohlc_df.index = ohlc["time"]
    ohlc_df = ohlc_df.apply(pd.to_numeric)
    ohlc_df.columns=["Open","High","Low","Close","Volume"]
    print("took data for ",instrument)
    return ohlc_df

In [9]:
def trade_signal(instrument,l_s):
    

    
    ohlc_dict=pd.DataFrame()
    ohlc_dict=candles(instrument)
    
    
    ohlc_dict.columns=["open","high","low","close","volume"]
    ohlc_dict["fisher 20"]=TA.FISH(ohlc_dict,20)
    ohlc_dict["fisher 10"]=TA.FISH(ohlc_dict,10)    
    ohlc_dict["RSI 16"]=TA.RSI(ohlc_dict,16)
    ohlc_dict["RSI 21"]=TA.RSI(ohlc_dict,21)
    ohlc_dict["MACD macd line"]=MACD(ohlc_dict,12,26,9)["MACD"]
    ohlc_dict["MACD signal line"]=MACD(ohlc_dict,12,26,9)["Signal"]

    signal=""
    if l_s=="":
        if signal=="":
            if ohlc_dict["fisher 10"][-1]>0 and ohlc_dict["fisher 10"][-2]<0 and ohlc_dict["RSI 16"][-1]>50: 
                signal="buy"
                
                
         
                
               



            elif (ohlc_dict["fisher 20"][-1]<0 and ohlc_dict["fisher 20"][-1-1]>0) and ohlc_dict["MACD macd line"][-1]<ohlc_dict["MACD signal line"][-1] and ohlc_dict["RSI 21"][-1]<42.5:
                signal="sell"
                        
      

        elif l_s=="long":
            if (ohlc_dict["fisher 20"][-1]<0.5 and ohlc_dict["fisher 20"][-1-1]>0.5):

                signal="squareoffsell"
               

           
        
        
        elif l_s=="short":
            if (ohlc_dict["fisher 20"][-1]>-1 and ohlc_dict["fisher 20"][-1-1]<-1):
                

                signal="squareoffbuy"


    return signal          




In [10]:
def long_short(instrument):
    account_id = "101-002-19512089-001"
    r=trades.OpenTrades(accountID=account_id)
    l_s=""
    open_trades=client.request(r)
  
    for i in range(len(open_trades)):
        if open_trades["trades"][i]["instrument"]==instrument and float(open_trades["trades"][i]["currentUnits"])>0:
            l_s="long"    

        elif open_trades["trades"][i]["instrument"]==instrument and float(open_trades["trades"][i]["currentUnits"])<0:
            l_s="short" 

    return l_s   

    

In [11]:
def main():
    global tickers,investment,risk
    for ticker in tickers:
        print("\n \n analyzzing for ",ticker)

          
        l_s= long_short(ticker)
        print(l_s)
        signal=trade_signal(ticker,l_s)
        if signal=="buy":
            market_order(ticker,investment,risk)
            print("New long position initiated for ",ticker)

        elif signal=="sell":
            market_order(ticker,-1*investment,risk)
            print("New short position initiated for ", ticker)

        elif signal=="squareoffbuy":
            market_order1(ticker,investment)

        elif signal=="squareoffsell":
            market_order2(ticker,-1*investment)

    
   
            



In [12]:
start_time=time.time()
timeout=start_time+60*60*1
while time.time()<=timeout:
    try: 
        print("passthrough at ",time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
        main()
        time.sleep(300-(time.time()-start_time)%300)

    except KeyboardInterrupt:
        print("keyboard interuption ....... exiting")


passthrough at  2021-06-13 00:41:24

 
 analyzzing for  EUR_USD

took data for  EUR_USD

 
 analyzzing for  AUD_USD
long
took data for  AUD_USD

 
 analyzzing for  GBP_USD

took data for  GBP_USD

 
 analyzzing for  USD_JPY

took data for  USD_JPY

 
 analyzzing for  EUR_JPY
long
took data for  EUR_JPY
passthrough at  2021-06-13 00:46:24

 
 analyzzing for  EUR_USD


ReadTimeout: HTTPSConnectionPool(host='api-fxpractice.oanda.com', port=443): Read timed out. (read timeout=None)